In [2]:
import glob
import typing

import numpy as np
import pandas as pd
from pycaret import classification
from scipy import signal, stats


# Utilities


In [3]:
# training patterns, names
ds_paths = (
    ("./train/*Thigh_acc_mag.csv", "thigh_acc_mag"),
    ("./train/*Thigh_acc_x.csv", "thigh_acc_x"),
    ("./train/*Thigh_acc_y.csv", "thigh_acc_y"),
    ("./train/*Thigh_acc_z.csv", "thigh_acc_z"),
    ("./train/*Thigh_gyr_mag.csv", "thigh_gyr_mag"),
    ("./train/*Thigh_gyr_x.csv", "thigh_gyr_x"),
    ("./train/*Thigh_gyr_y.csv", "thigh_gyr_y"),
    ("./train/*Thigh_gyr_z.csv", "thigh_gyr_z"),
    ("./train/*Wrist_acc_mag.csv", "wrist_acc_mag"),
    ("./train/*Wrist_acc_x.csv", "wrist_acc_x"),
    ("./train/*Wrist_acc_y.csv", "wrist_acc_y"),
    ("./train/*Wrist_acc_z.csv", "wrist_acc_z"),
    ("./train/*Wrist_gyr_mag.csv", "wrist_gyr_mag"),
    ("./train/*Wrist_gyr_x.csv", "wrist_gyr_x"),
    ("./train/*Wrist_gyr_y.csv", "wrist_gyr_y"),
    ("./train/*Wrist_gyr_z.csv", "wrist_gyr_z"),
)
# submission paths, names
ds_submission_paths = (
    ("./test/Thigh_acc_mag.csv", "thigh_acc_mag"),
    ("./test/Thigh_acc_x.csv", "thigh_acc_x"),
    ("./test/Thigh_acc_y.csv", "thigh_acc_y"),
    ("./test/Thigh_acc_z.csv", "thigh_acc_z"),
    ("./test/Thigh_gyr_mag.csv", "thigh_gyr_mag"),
    ("./test/Thigh_gyr_x.csv", "thigh_gyr_x"),
    ("./test/Thigh_gyr_y.csv", "thigh_gyr_y"),
    ("./test/Thigh_gyr_z.csv", "thigh_gyr_z"),
    ("./test/Wrist_acc_mag.csv", "wrist_acc_mag"),
    ("./test/Wrist_acc_x.csv", "wrist_acc_x"),
    ("./test/Wrist_acc_y.csv", "wrist_acc_y"),
    ("./test/Wrist_acc_z.csv", "wrist_acc_z"),
    ("./test/Wrist_gyr_mag.csv", "wrist_gyr_mag"),
    ("./test/Wrist_gyr_x.csv", "wrist_gyr_x"),
    ("./test/Wrist_gyr_y.csv", "wrist_gyr_y"),
    ("./test/Wrist_gyr_z.csv", "wrist_gyr_z"),
)
# columns to exclude from certain dataframe operations
exclude_cols = [
    "index",
    "group",
    "activity_id",
    "mean",
    "stddev",
    "median",
    "sum",
    "min",
    "max",
    "quantile_1",
    "quantile_2",
    "quantile_3",
    "skewness",
    "kurtosis",
    "trend",
    "root_mean_square",
    "range",
    "integral",
    "mean_crossing_rate",
    "number_of_peaks",
    "average_peak_height",
    "peak_to_average_power_ratio",
    *[f"freq_mag_{i + 1}" for i in range(3)],
    *[f"mag_{i + 1}" for i in range(10)],
    "energy",
    "entropy",
    *[f"pds_bin_{i + 1}" for i in range(10)],
    "power_spectral_density_skewness",
    "power_spectral_density_kurtosis",
]
train_subjects = ("s0", "s1", "s4", "s5", "s6", "s8")
test_subjects = ("s7", "s9")


def __calculate_psd__(row: pd.Series):
    return signal.welch(row, fs=50.0, nperseg=100)


def __calculate_psd_magnitudes__(row: pd.Series, index: int):
    f, Pxx_den = __calculate_psd__(row=row)
    index_of_ith_max = Pxx_den.argsort()[-index]
    return f[index_of_ith_max], Pxx_den[index_of_ith_max]


def __get_freq_mag_i__(row: pd.Series, index: int):
    return __calculate_psd_magnitudes__(row=row, index=index)[0]


def __get_mag_i__(row: pd.Series, index: int):
    return __calculate_psd_magnitudes__(row=row, index=index)[1]


def __calculate_psd_histogram_bin_i__(row: pd.Series, index: int):
    f, Pxx_den = __calculate_psd__(row=row)
    return np.sum(np.square(np.abs(Pxx_den[5 * index : 5 * (index + 1)]))) / np.sum(
        np.square(np.abs(Pxx_den))
    )


def merge_data(
    paths: typing.Tuple[typing.Tuple[str, str]], subjects=tuple()
) -> pd.DataFrame:
    # function to merge everything given a list of tuples (patterns, names)
    dfs = list()
    for pattern, name in paths:
        sensors_dfs = list()
        # read all files matching that pattern (one per subject)
        for ds_path in glob.glob(pattern):
            sensors_dfs.append(pd.read_csv(ds_path))
        # if subject_ids were passed then create group column
        for i, subj_id in enumerate(subjects):
            sensors_dfs[i]["group"] = subj_id

        # merge everything into a single dataframe
        df_merged = pd.concat(sensors_dfs)

        df_stats = df_merged.copy()
        original_columns = [c for c in df_stats.columns if c not in exclude_cols]
        # compute mean
        df_stats["mean"] = df_stats[original_columns].mean(axis=1)
        # compute stddev
        df_stats["stddev"] = df_stats[original_columns].std(axis=1)
        # compute median
        df_stats["median"] = df_stats[original_columns].median(axis=1)
        # compute sum
        df_stats["sum"] = df_stats[original_columns].sum(axis=1)
        # compute min
        df_stats["min"] = df_stats[original_columns].min(axis=1)
        # compute max
        df_stats["max"] = df_stats[original_columns].max(axis=1)
        # compute first quartile
        df_stats["quartile_1"] = df_stats[original_columns].quantile(q=0.25, axis=1)
        # compute second quartile
        df_stats["quartile_2"] = df_stats[original_columns].quantile(q=0.5, axis=1)
        # compute third quartile
        df_stats["quartile_3"] = df_stats[original_columns].quantile(q=0.75, axis=1)
        # compute skewness
        df_stats["skewness"] = df_stats[original_columns].skew(axis=1)
        # compute kurtosis
        df_stats["kurtosis"] = df_stats[original_columns].kurtosis(axis=1)
        # compute trend (positive if increasing, negative if decreasing, ~0 if constant)
        df_stats["trend"] = df_stats[original_columns].apply(
            lambda row: np.polyfit(range(len(original_columns)), row.tolist(), 1)[0],
            axis=1,
        )
        # compute root mean square
        df_stats["root_mean_square"] = (
            df_stats[original_columns].pow(2).mean(axis=1).pow(1 / 2)
        )
        # compute range
        df_stats["range"] = (df_stats["max"] - df_stats["min"]).abs()
        # compute integral
        df_stats["integral"] = df_stats[original_columns].apply(np.trapz, axis=1)
        # compute mean crossing rate
        df_stats["mean_crossing_rate"] = df_stats.apply(
            lambda row: len(np.nonzero(np.diff(row[original_columns] > row["mean"]))[0])
            / len(row[original_columns]),
            axis=1,
        )
        # compute peak-to-average power ratio in decibels
        df_stats["peak_to_average_power_ratio"] = df_stats.apply(
            lambda row: 10
            * np.log10(
                pow(max(abs(row["min"]), abs(row["max"])), 2)
                / pow(row["root_mean_square"], 2)
            ),
            axis=1,
        )
        # compute 3 largest magnitudes
        for i in range(3):
            df_stats[f"freq_mag_{i + 1}"] = df_stats[original_columns].apply(
                lambda row: __get_freq_mag_i__(row=row, index=i + 1), axis=1
            )
            df_stats[f"mag_{i + 1}"] = df_stats[original_columns].apply(
                lambda row: __get_mag_i__(row=row, index=i + 1), axis=1
            )
        # compute energy
        df_stats["energy"] = np.sum(
            np.square(np.abs(np.fft.fft(df_stats[original_columns], axis=1))), axis=1
        ).reshape((-1, 1)) / len(original_columns)
        # compute entropy
        df_stats["entropy"] = stats.entropy(
            np.abs(np.fft.fft(df_stats[original_columns], axis=1)), axis=1
        ).reshape((-1, 1))
        # compute 10 bin histogram for power spectral density
        for i in range(10):
            df_stats[f"pds_bin_{i + 1}"] = df_stats[original_columns].apply(
                lambda row: __calculate_psd_histogram_bin_i__(row=row, index=i), axis=1
            )
        # compute skewness of power spectral density
        df_stats["power_spectral_density_skewness"] = df_stats[original_columns].apply(
            lambda row: stats.skew(__calculate_psd__(row=row)[1]), axis=1
        )
        # compute kurtosis of power spectral density
        df_stats["power_spectral_density_kurtosis"] = df_stats[original_columns].apply(
            lambda row: stats.kurtosis(__calculate_psd__(row=row)[1]), axis=1
        )

        # drop original columns
        df_stats.drop(original_columns, axis=1, inplace=True)
        # rename only computed columns
        df_stats.columns = [
            f"{col}_{name}" if col not in ("index", "group") else col
            for col in df_stats.columns
        ]
        # append processed dataframe
        dfs.append(df_stats.copy())
    # return the merge of all processed dataframes
    return pd.concat(dfs, axis=1)


# Load Data


In [4]:
X_full = merge_data(paths=ds_paths, subjects=train_subjects + test_subjects)
X_full


,index,group,mean_thigh_acc_mag,stddev_thigh_acc_mag,median_thigh_acc_mag,sum_thigh_acc_mag,min_thigh_acc_mag,max_thigh_acc_mag,quartile_1_thigh_acc_mag,quartile_2_thigh_acc_mag,quartile_3_thigh_acc_mag,skewness_thigh_acc_mag,kurtosis_thigh_acc_mag,trend_thigh_acc_mag,root_mean_square_thigh_acc_mag,range_thigh_acc_mag,integral_thigh_acc_mag,mean_crossing_rate_thigh_acc_mag,peak_to_average_power_ratio_thigh_acc_mag,freq_mag_1_thigh_acc_mag,mag_1_thigh_acc_mag,freq_mag_2_thigh_acc_mag,mag_2_thigh_acc_mag,freq_mag_3_thigh_acc_mag,mag_3_thigh_acc_mag,energy_thigh_acc_mag,entropy_thigh_acc_mag,pds_bin_1_thigh_acc_mag,pds_bin_2_thigh_acc_mag,pds_bin_3_thigh_acc_mag,pds_bin_4_thigh_acc_mag,pds_bin_5_thigh_acc_mag,pds_bin_6_thigh_acc_mag,pds_bin_7_thigh_acc_mag,pds_bin_8_thigh_acc_mag,pds_bin_9_thigh_acc_mag,pds_bin_10_thigh_acc_mag,power_spectral_density_skewness_thigh_acc_mag,power_spectral_density_kurtosis_thigh_acc_mag,index,group,mean_thigh_acc_x,stddev_thigh_acc_x,median_thigh_acc_x,sum_thigh_acc_x,min_thigh_acc_x,max_thigh_acc_x,quartile_1_thigh_acc_x,quartile_2_thigh_acc_x,quartile_3_thigh_acc_x,skewness_thigh_acc_x,kurtosis_thigh_acc_x,trend_thigh_acc_x,root_mean_square_thigh_acc_x,range_thigh_acc_x,integral_thigh_acc_x,mean_crossing_rate_thigh_acc_x,peak_to_average_power_ratio_thigh_acc_x,freq_mag_1_thigh_acc_x,mag_1_thigh_acc_x,freq_mag_2_thigh_acc_x,mag_2_thigh_acc_x,freq_mag_3_thigh_acc_x,mag_3_thigh_acc_x,energy_thigh_acc_x,entropy_thigh_acc_x,pds_bin_1_thigh_acc_x,pds_bin_2_thigh_acc_x,pds_bin_3_thigh_acc_x,pds_bin_4_thigh_acc_x,pds_bin_5_thigh_acc_x,pds_bin_6_thigh_acc_x,pds_bin_7_thigh_acc_x,pds_bin_8_thigh_acc_x,pds_bin_9_thigh_acc_x,pds_bin_10_thigh_acc_x,power_spectral_density_skewness_thigh_acc_x,power_spectral_density_kurtosis_thigh_acc_x,index,group,mean_thigh_acc_y,stddev_thigh_acc_y,median_thigh_acc_y,sum_thigh_acc_y,min_thigh_acc_y,max_thigh_acc_y,quartile_1_thigh_acc_y,quartile_2_thigh_acc_y,quartile_3_thigh_acc_y,skewness_thigh_acc_y,kurtosis_thigh_acc_y,trend_thigh_acc_y,root_mean_square_thigh_acc_y,range_thigh_acc_y,integral_thigh_acc_y,mean_crossing_rate_thigh_acc_y,peak_to_average_power_ratio_thigh_acc_y,freq_mag_1_thigh_acc_y,mag_1_thigh_acc_y,freq_mag_2_thigh_acc_y,mag_2_thigh_acc_y,freq_mag_3_thigh_acc_y,mag_3_thigh_acc_y,energy_thigh_acc_y,entropy_thigh_acc_y,pds_bin_1_thigh_acc_y,pds_bin_2_thigh_acc_y,pds_bin_3_thigh_acc_y,pds_bin_4_thigh_acc_y,pds_bin_5_thigh_acc_y,pds_bin_6_thigh_acc_y,pds_bin_7_thigh_acc_y,pds_bin_8_thigh_acc_y,pds_bin_9_thigh_acc_y,pds_bin_10_thigh_acc_y,power_spectral_density_skewness_thigh_acc_y,power_spectral_density_kurtosis_thigh_acc_y,index,group,mean_thigh_acc_z,stddev_thigh_acc_z,median_thigh_acc_z,sum_thigh_acc_z,min_thigh_acc_z,max_thigh_acc_z,quartile_1_thigh_acc_z,quartile_2_thigh_acc_z,quartile_3_thigh_acc_z,skewness_thigh_acc_z,kurtosis_thigh_acc_z,trend_thigh_acc_z,root_mean_square_thigh_acc_z,range_thigh_acc_z,integral_thigh_acc_z,mean_crossing_rate_thigh_acc_z,peak_to_average_power_ratio_thigh_acc_z,freq_mag_1_thigh_acc_z,mag_1_thigh_acc_z,freq_mag_2_thigh_acc_z,mag_2_thigh_acc_z,freq_mag_3_thigh_acc_z,mag_3_thigh_acc_z,energy_thigh_acc_z,entropy_thigh_acc_z,pds_bin_1_thigh_acc_z,pds_bin_2_thigh_acc_z,pds_bin_3_thigh_acc_z,pds_bin_4_thigh_acc_z,pds_bin_5_thigh_acc_z,pds_bin_6_thigh_acc_z,pds_bin_7_thigh_acc_z,pds_bin_8_thigh_acc_z,pds_bin_9_thigh_acc_z,pds_bin_10_thigh_acc_z,power_spectral_density_skewness_thigh_acc_z,power_spectral_density_kurtosis_thigh_acc_z,index,group,mean_thigh_gyr_mag,stddev_thigh_gyr_mag,median_thigh_gyr_mag,sum_thigh_gyr_mag,min_thigh_gyr_mag,max_thigh_gyr_mag,quartile_1_thigh_gyr_mag,quartile_2_thigh_gyr_mag,quartile_3_thigh_gyr_mag,skewness_thigh_gyr_mag,kurtosis_thigh_gyr_mag,trend_thigh_gyr_mag,root_mean_square_thigh_gyr_mag,range_thigh_gyr_mag,integral_thigh_gyr_mag,mean_crossing_rate_thigh_gyr_mag,peak_to_average_power_ratio_thigh_gyr_mag,freq_mag_1_thigh_gyr_mag,mag_1_thigh_gyr_mag,freq_mag_2_thigh_gyr_mag,mag_2_thigh_gyr_mag,fr

In [5]:
y_dfs = list()
for x in glob.glob("./train/*Thigh_activity_id.csv"):
    # thigh or wrist it doesn't matter, they are the same
    y_dfs.append(pd.read_csv(x))
y_full = pd.concat(y_dfs)
y_full


,index,activity_id
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
3425,3425,3
3426,3426,3
3427,3427,5
3428,3428,5


In [6]:
# merge data and label
df_full = pd.concat((X_full, y_full), axis=1)
idx = df_full.iloc[:, 0]
grp = df_full.iloc[:, 1]
# drop multiple columns named "index" and "group"
df_full.drop(["index"], axis=1, inplace=True)
# drop multiple columns named group
df_full.drop(["group"], axis=1, inplace=True)
# recreate index and group columns
df_full.insert(loc=0, column="index", value=idx)
df_full.insert(loc=1, column="group", value=grp)
# drop index
df_full.reset_index(drop=True, inplace=True)
df_full.to_csv("df_full.csv", index=False)
df_full


,index,group,mean_thigh_acc_mag,stddev_thigh_acc_mag,median_thigh_acc_mag,sum_thigh_acc_mag,min_thigh_acc_mag,max_thigh_acc_mag,quartile_1_thigh_acc_mag,quartile_2_thigh_acc_mag,quartile_3_thigh_acc_mag,skewness_thigh_acc_mag,kurtosis_thigh_acc_mag,trend_thigh_acc_mag,root_mean_square_thigh_acc_mag,range_thigh_acc_mag,integral_thigh_acc_mag,mean_crossing_rate_thigh_acc_mag,peak_to_average_power_ratio_thigh_acc_mag,freq_mag_1_thigh_acc_mag,mag_1_thigh_acc_mag,freq_mag_2_thigh_acc_mag,mag_2_thigh_acc_mag,freq_mag_3_thigh_acc_mag,mag_3_thigh_acc_mag,energy_thigh_acc_mag,entropy_thigh_acc_mag,pds_bin_1_thigh_acc_mag,pds_bin_2_thigh_acc_mag,pds_bin_3_thigh_acc_mag,pds_bin_4_thigh_acc_mag,pds_bin_5_thigh_acc_mag,pds_bin_6_thigh_acc_mag,pds_bin_7_thigh_acc_mag,pds_bin_8_thigh_acc_mag,pds_bin_9_thigh_acc_mag,pds_bin_10_thigh_acc_mag,power_spectral_density_skewness_thigh_acc_mag,power_spectral_density_kurtosis_thigh_acc_mag,mean_thigh_acc_x,stddev_thigh_acc_x,median_thigh_acc_x,sum_thigh_acc_x,min_thigh_acc_x,max_thigh_acc_x,quartile_1_thigh_acc_x,quartile_2_thigh_acc_x,quartile_3_thigh_acc_x,skewness_thigh_acc_x,kurtosis_thigh_acc_x,trend_thigh_acc_x,root_mean_square_thigh_acc_x,range_thigh_acc_x,integral_thigh_acc_x,mean_crossing_rate_thigh_acc_x,peak_to_average_power_ratio_thigh_acc_x,freq_mag_1_thigh_acc_x,mag_1_thigh_acc_x,freq_mag_2_thigh_acc_x,mag_2_thigh_acc_x,freq_mag_3_thigh_acc_x,mag_3_thigh_acc_x,energy_thigh_acc_x,entropy_thigh_acc_x,pds_bin_1_thigh_acc_x,pds_bin_2_thigh_acc_x,pds_bin_3_thigh_acc_x,pds_bin_4_thigh_acc_x,pds_bin_5_thigh_acc_x,pds_bin_6_thigh_acc_x,pds_bin_7_thigh_acc_x,pds_bin_8_thigh_acc_x,pds_bin_9_thigh_acc_x,pds_bin_10_thigh_acc_x,power_spectral_density_skewness_thigh_acc_x,power_spectral_density_kurtosis_thigh_acc_x,mean_thigh_acc_y,stddev_thigh_acc_y,median_thigh_acc_y,sum_thigh_acc_y,min_thigh_acc_y,max_thigh_acc_y,quartile_1_thigh_acc_y,quartile_2_thigh_acc_y,quartile_3_thigh_acc_y,skewness_thigh_acc_y,kurtosis_thigh_acc_y,trend_thigh_acc_y,root_mean_square_thigh_acc_y,range_thigh_acc_y,integral_thigh_acc_y,mean_crossing_rate_thigh_acc_y,peak_to_average_power_ratio_thigh_acc_y,freq_mag_1_thigh_acc_y,mag_1_thigh_acc_y,freq_mag_2_thigh_acc_y,mag_2_thigh_acc_y,freq_mag_3_thigh_acc_y,mag_3_thigh_acc_y,energy_thigh_acc_y,entropy_thigh_acc_y,pds_bin_1_thigh_acc_y,pds_bin_2_thigh_acc_y,pds_bin_3_thigh_acc_y,pds_bin_4_thigh_acc_y,pds_bin_5_thigh_acc_y,pds_bin_6_thigh_acc_y,pds_bin_7_thigh_acc_y,pds_bin_8_thigh_acc_y,pds_bin_9_thigh_acc_y,pds_bin_10_thigh_acc_y,power_spectral_density_skewness_thigh_acc_y,power_spectral_density_kurtosis_thigh_acc_y,mean_thigh_acc_z,stddev_thigh_acc_z,median_thigh_acc_z,sum_thigh_acc_z,min_thigh_acc_z,max_thigh_acc_z,quartile_1_thigh_acc_z,quartile_2_thigh_acc_z,quartile_3_thigh_acc_z,skewness_thigh_acc_z,kurtosis_thigh_acc_z,trend_thigh_acc_z,root_mean_square_thigh_acc_z,range_thigh_acc_z,integral_thigh_acc_z,mean_crossing_rate_thigh_acc_z,peak_to_average_power_ratio_thigh_acc_z,freq_mag_1_thigh_acc_z,mag_1_thigh_acc_z,freq_mag_2_thigh_acc_z,mag_2_thigh_acc_z,freq_mag_3_thigh_acc_z,mag_3_thigh_acc_z,energy_thigh_acc_z,entropy_thigh_acc_z,pds_bin_1_thigh_acc_z,pds_bin_2_thigh_acc_z,pds_bin_3_thigh_acc_z,pds_bin_4_thigh_acc_z,pds_bin_5_thigh_acc_z,pds_bin_6_thigh_acc_z,pds_bin_7_thigh_acc_z,pds_bin_8_thigh_acc_z,pds_bin_9_thigh_acc_z,pds_bin_10_thigh_acc_z,power_spectral_density_skewness_thigh_acc_z,power_spectral_density_kurtosis_thigh_acc_z,mean_thigh_gyr_mag,stddev_thigh_gyr_mag,median_thigh_gyr_mag,sum_thigh_gyr_mag,min_thigh_gyr_mag,max_thigh_gyr_mag,quartile_1_thigh_gyr_mag,quartile_2_thigh_gyr_mag,quartile_3_thigh_gyr_mag,skewness_thigh_gyr_mag,kurtosis_thigh_gyr_mag,trend_thigh_gyr_mag,root_mean_square_thigh_gyr_mag,range_thigh_gyr_mag,integral_thigh_gyr_mag,mean_crossing_rate_thigh_gyr_mag,peak_to_average_power_ratio_thigh_gyr_mag,freq_mag_1_thigh_gyr_mag,mag_1_thigh_gyr_mag,freq_mag_2_thigh_gyr_mag,mag_2_thigh_gyr_mag,freq_mag_3_thigh_gyr_mag,mag_3_thigh_gyr_mag,energ

In [7]:
df_train = df_full[df_full["group"].isin(train_subjects)]
df_test = df_full[df_full["group"].isin(test_subjects)]


# Setup PyCaret


In [8]:
cf_setup = classification.setup(
    data=df_train,
    test_data=df_test,
    target="activity_id",
    preprocess=True,
    ignore_features=["index", "group"],
    # feature_selection=True,
    # feature_selection_threshold=0.7,
    handle_unknown_categorical=False,
    remove_multicollinearity=True,
    session_id=666,
    fold_strategy="groupkfold",
    fold_groups="group",
    fold=5,
    silent=True,
)


,Description,Value
0,session_id,666
1,Target,activity_id
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(22910, 595)"
5,Missing Values,False
6,Numeric Features,592
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


# Train Models

In [9]:
best_models = classification.compare_models(sort="F1", n_select=20)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.8920,0.9874,0.8171,0.9032,0.8863,0.8739,0.8764,1.5580
lda,Linear Discriminant Analysis,0.8726,0.9857,0.8112,0.8832,0.8703,0.8521,0.8540,1.2560
rf,Random Forest Classifier,0.8704,0.9854,0.8244,0.8940,0.8658,0.8502,0.8550,1.9900
lightgbm,Light Gradient Boosting Machine,0.8623,0.9866,0.8310,0.8919,0.8555,0.8414,0.8476,19.1940
ridge,Ridge Classifier,0.8542,0.0000,0.7378,0.8672,0.8408,0.8296,0.8324,0.1820
gbc,Gradient Boosting Classifier,0.8484,0.9843,0.8087,0.8751,0.8404,0.8253,0.8319,318.2220
dt,Decision Tree Classifier,0.8092,0.8925,0.7520,0.8127,0.8039,0.7790,0.7810,2.1540
qda,Quadratic Discriminant Analysis,0.7398,0.9318,0.6504,0.7596,0.7368,0.6989,0.7034,0.7000
ada,Ada Boost Classifier,0.4709,0.7933,0.3923,0.4584,0.3699,0.3792,0.4373,6.7800
nb,Naive Bayes,0.3343,0.7915,0.3760,0.4125,0.3126,0.2604,0.2972,0.7820


# Tune Best 3 Models

In [10]:
tuned_best_0 = classification.tune_model(
    best_models[0], optimize="F1", choose_better=True
)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8798,0.9835,0.8412,0.8912,0.8804,0.8612,0.8620
1,0.8600,0.9850,0.8489,0.8916,0.8647,0.8390,0.8419
2,0.7299,0.9859,0.8029,0.8281,0.7254,0.6943,0.7132
3,0.9010,0.9909,0.8478,0.9092,0.8860,0.8847,0.8880
4,0.8335,0.9761,0.7983,0.8630,0.8233,0.8049,0.8103
Mean,0.8409,0.9843,0.8278,0.8766,0.8360,0.8168,0.8231
SD,0.0598,0.0048,0.0224,0.0284,0.0595,0.0667,0.0605


In [11]:
tuned_best_1 = classification.tune_model(
    best_models[1], optimize="F1", choose_better=True
)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7974,0.9777,0.7105,0.8051,0.7790,0.7616,0.7673
1,0.8107,0.9783,0.7889,0.8748,0.8258,0.7835,0.7885
2,0.8619,0.9824,0.8486,0.8911,0.8658,0.8409,0.8441
3,0.8548,0.9874,0.7790,0.8671,0.8498,0.8301,0.8331
4,0.7746,0.9791,0.7021,0.8058,0.7677,0.7389,0.7467
Mean,0.8199,0.9810,0.7658,0.8488,0.8176,0.7910,0.7960
SD,0.0336,0.0036,0.0542,0.0362,0.0385,0.0391,0.0374


In [12]:
tuned_best_2 = classification.tune_model(
    best_models[2], optimize="F1", choose_better=True
)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8499,0.9758,0.7793,0.8709,0.8471,0.8262,0.8277
1,0.8753,0.9804,0.8353,0.8884,0.8766,0.8559,0.8572
2,0.6841,0.9752,0.7512,0.7856,0.6764,0.6435,0.6640
3,0.9220,0.9905,0.8565,0.9241,0.9197,0.9093,0.9100
4,0.8301,0.9705,0.7795,0.8541,0.8205,0.8018,0.8061
Mean,0.8323,0.9785,0.8004,0.8646,0.8280,0.8073,0.8130
SD,0.0802,0.0068,0.0392,0.0458,0.0827,0.0894,0.0823


# Load Submission Data


In [ ]:
X_submission = merge_data(paths=ds_submission_paths)
X_submission.drop(["index"], axis=1, inplace=True)


In [13]:
# select only columns that feature selection kept
X_submission = X_submission[classification.get_config("X").columns]
X_submission


,min_thigh_acc_mag,quartile_1_thigh_acc_mag,skewness_thigh_acc_mag,kurtosis_thigh_acc_mag,trend_thigh_acc_mag,mean_crossing_rate_thigh_acc_mag,peak_to_average_power_ratio_thigh_acc_mag,freq_mag_1_thigh_acc_mag,mag_1_thigh_acc_mag,freq_mag_2_thigh_acc_mag,...,pds_bin_2_wrist_gyr_z,pds_bin_3_wrist_gyr_z,pds_bin_4_wrist_gyr_z,pds_bin_5_wrist_gyr_z,pds_bin_6_wrist_gyr_z,pds_bin_7_wrist_gyr_z,pds_bin_8_wrist_gyr_z,pds_bin_9_wrist_gyr_z,pds_bin_10_wrist_gyr_z,power_spectral_density_kurtosis_wrist_gyr_z
0,0.972603,0.995347,-0.510562,0.167131,0.000012,0.49,0.191898,7.0,0.000027,7.5,...,7.317898e-02,3.400374e-02,2.312178e-02,1.361731e-01,2.218040e-01,5.907561e-02,2.751008e-01,1.152992e-01,1.767145e-02,0.906143
1,0.978905,0.995038,0.034116,-0.596567,0.000045,0.49,0.199249,10.5,0.000018,9.0,...,2.486642e-08,1.274255e-08,1.336190e-08,1.812091e-08,2.293336e-08,5.887719e-09,7.913693e-09,1.111403e-08,1.543777e-08,29.587885
2,0.976057,0.993388,0.212561,-0.047335,0.000035,0.57,0.312602,19.5,0.000026,12.5,...,2.377537e-02,9.700664e-02,4.412821e-03,1.790189e-03,3.729289e-02,3.048025e-03,5.369266e-02,1.872110e-02,4.619247e-03,29.884080
3,0.970288,0.993147,-0.308402,-0.443711,0.000012,0.52,0.216135,15.5,0.000017,23.0,...,1.235183e-01,1.947167e-01,1.479873e-02,4.121755e-02,1.298936e-02,9.846761e-03,1.004514e-01,2.790528e-03,1.190401e-01,2.970785
4,0.968164,0.987913,-0.061873,-0.625855,-0.000018,0.51,0.240117,13.5,0.000040,13.0,...,4.807169e-02,5.190343e-02,2.909786e-02,4.149845e-02,2.862930e-01,1.160947e-01,2.491636e-02,1.440681e-02,4.915570e-02,6.465115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7745,0.961100,1.008184,1.680622,6.641719,0.000190,0.45,0.924689,3.5,0.000022,16.5,...,5.967231e-02,8.658445e-03,2.431749e-04,1.768809e-04,6.697691e-04,1.759973e-04,1.479122e-05,1.488494e-04,1.098750e-04,6.116513
7746,0.911579,0.987228,1.653757,2.353944,0.002048,0.25,2.471036,0.5,0.001842,0.0,...,4.584347e-01,7.683296e-05,2.174411e-05,2.769831e-06,1.305286e-06,7.594126e-05,1.667938e-05,3.654013e-05,1.505066e-04,8.885343
7747,0.729166,0.942012,0.061376,0.446306,0.000440,0.39,2.110815,2.0,0.003047,3.5,...,2.031897e-01,4.472687e-04,9.002936e-06,4.019146e-06,5.810384e-06,2.376446e-06,1.285125e-05,6.143536e-06,3.624163e-06,8.433608
7748,0.973979,0.994961,0.374718,-0.368472,0.000008,0.42,0.283030,10.5,0.000026,24.5,...,5.367880e-03,7.148306e-07,3.319797e-07,7.649998e-08,2.489584e-06,3.509299e-06,4.957502e-06,4.503048e-06,2.507117e-05,31.419031


# Predict and Save


In [14]:
df_tuned_best_0 = pd.DataFrame()
df_tuned_best_0["index"] = X_submission.index
df_tuned_best_0["activity_id"] = tuned_best_0.predict(X_submission)
df_tuned_best_0.to_csv("./submissions/tuned_best_0.csv", index=False)
df_tuned_best_1 = pd.DataFrame()
df_tuned_best_1["index"] = X_submission.index
df_tuned_best_1["activity_id"] = tuned_best_1.predict(X_submission)
df_tuned_best_1.to_csv("./submissions/tuned_best_1.csv", index=False)
df_tuned_best_2 = pd.DataFrame()
df_tuned_best_2["index"] = X_submission.index
df_tuned_best_2["activity_id"] = tuned_best_2.predict(X_submission)
df_tuned_best_2.to_csv("./submissions/tuned_best_2.csv", index=False)


# Try Stacking Models


In [15]:
stacked_model = classification.stack_models(
    best_models[:3], choose_better=True, optimize="F1"
)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.3900,0.7295,0.2731,0.2553,0.2847,0.2373,0.2881
1,0.3406,0.7187,0.1804,0.1907,0.2268,0.1714,0.2143
2,0.3302,0.7328,0.1778,0.2308,0.2207,0.1533,0.1994
3,0.3396,0.7347,0.1794,0.2236,0.2269,0.1698,0.2158
4,0.3394,0.7171,0.2095,0.2238,0.2419,0.1739,0.2143
Mean,0.3480,0.7266,0.2041,0.2248,0.2402,0.1812,0.2264
SD,0.0213,0.0073,0.0365,0.0206,0.0233,0.0290,0.0314


# Train Final Models on Train and Validation Data


In [17]:
final_0 = classification.finalize_model(best_models[0])


In [18]:
final_2 = classification.finalize_model(tuned_best_2)


# Try Blending Finalised Models


In [19]:
blended_model = classification.blend_models(
    [final_0, final_2], choose_better=True, optimize="F1"
)


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8991,0.9847,0.8268,0.9039,0.8938,0.8825,0.8842
1,0.9052,0.9888,0.8590,0.9073,0.9037,0.8900,0.8905
2,0.8737,0.9878,0.8336,0.8864,0.8740,0.8537,0.8556
3,0.9157,0.9931,0.8196,0.9292,0.9082,0.9015,0.9041
4,0.8823,0.9833,0.8158,0.8991,0.8729,0.8620,0.8658
Mean,0.8952,0.9875,0.8310,0.9052,0.8905,0.8779,0.8801
SD,0.0152,0.0034,0.0153,0.0139,0.0147,0.0177,0.0174


# Predict and Save


In [20]:
df_final_0 = pd.DataFrame()
df_final_0["index"] = X_submission.index
df_final_0["activity_id"] = final_0.predict(X_submission)
df_final_0.to_csv("./submissions/final_0.csv", index=False)
df_final_2 = pd.DataFrame()
df_final_2["index"] = X_submission.index
df_final_2["activity_id"] = final_2.predict(X_submission)
df_final_2.to_csv("./submissions/final_2.csv", index=False)
df_blended_model = pd.DataFrame()
df_blended_model["index"] = X_submission.index
df_blended_model["activity_id"] = blended_model.predict(X_submission)
df_blended_model.to_csv("./submissions/blended_model.csv", index=False)
